#### Задание №1

In [1]:
from zipfile import ZipFile
from pathlib import Path
import json
import orjson
import pandas as pd
import numpy as np
import io
import sqlite3
from tqdm.notebook import tqdm

In [7]:
# pip install ipywidgets

In [22]:
# получаем список файлов в архиве
zip_file = ZipFile('okved_2.json.zip' , 'r')
list_files = [text_file.filename for text_file in zip_file.infolist()]

In [23]:
# чтение каждого файла в архиве без его распаковки
for i in list_files:
    with ZipFile('okved_2.json.zip' , 'r') as archive:
        content = pd.read_json(io.BytesIO(archive.read(i)),dtype={
                            'code':str,
                            'parent_code': str,
                            'section': str,
                            'name': str,
                            'comment': str})

In [24]:
content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2818 entries, 0 to 2817
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   code         2818 non-null   object
 1   parent_code  2818 non-null   object
 2   section      2818 non-null   object
 3   name         2818 non-null   object
 4   comment      2818 non-null   object
dtypes: object(5)
memory usage: 110.2+ KB


In [32]:
# создание БД
connection = sqlite3.connect('hw1.db')

In [33]:
content.to_sql('okved', 
             connection, 
             if_exists='append',
             index=False)

# Фиксируем изменения
connection.commit()
# проверка кол-ва записей в БД
pd.read_sql("SELECT COUNT(*) FROM okved", connection)

,COUNT(*)
0,2818


In [34]:
# Закрываем соединение
connection.close()

#### Задание №2

In [2]:
# Вариант 1. Обработка файлов по одному с распаковкой
pbar = tqdm()
black_list = []
df_full = pd.DataFrame(columns=['ogrn','inn','kpp','name','full_name','data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД'])
with ZipFile(r'C:\WORK\STUDY\MIPT\ДЗ\1\egrul.json.zip') as zipobj:
    file_names = zipobj.namelist()
    pbar.reset(total=len(file_names))
    for name in file_names:
        # Распаковываем один файл
        zipobj.extract(name)
        # Обрабатываем файл
        with io.open(name, 'r', encoding='utf-8') as read_file:
            json_str = read_file.read()
            df = pd.json_normalize(orjson.loads(json_str))
            if 'data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД' in df.columns.values.tolist():
                df = df[df['data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД'].str.contains('^61').fillna(False)][['ogrn','inn','kpp','name','full_name','data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД']]
                df = df.astype({'ogrn':str,'inn': str,'kpp': str,'name': str,'full_name': str,'data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД': str})
                df_full = pd.concat([df_full, df], ignore_index=True, sort=False)
            else:
                black_list.append(name)
        pbar.update()        
        # Удаляем файл
        path = Path(name)
        path.unlink()
pbar.refresh()
connection = sqlite3.connect('hw1.db')
df_full.to_sql('telecom_companies', 
                 connection, 
                 if_exists='append',
                 index=False)
with open('black_list.txt', 'w', encoding='UTF-8') as f:
    f.write(str(black_list))

0it [00:00, ?it/s]

In [ ]:
# Вариант 2. Обработка файлов по одному без его распаковки
pbar = tqdm()
black_list = []
df_full = pd.DataFrame(columns=['ogrn','inn','kpp','name','full_name','data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД'])
with ZipFile(r'C:\WORK\STUDY\MIPT\ДЗ\1\egrul.json.zip') as zipobj:
    file_names = zipobj.namelist()
    pbar.reset(total=len(file_names))
    for name in file_names:
        # Считываем файл из архива в bites, формируем в json, нормализуем json для Pandas
        df = pd.json_normalize(orjson.loads(zipobj.read(name)))
        if 'data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД' in df.columns.values.tolist():
            df = df[df['data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД'].str.contains('^61').fillna(False)][['ogrn','inn','kpp','name','full_name','data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД']]
            df = df.astype({'ogrn':str,'inn': str,'kpp': str,'name': str,'full_name': str,'data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД': str})
            df_full = pd.concat([df_full, df], ignore_index=True, sort=False)
        else:
            black_list.append(name) 
        pbar.update()                
pbar.refresh()

connection = sqlite3.connect('hw1.db')
df_full.to_sql('telecom_companies', 
                 connection, 
                 if_exists='append',
                 index=False)
with open('black_list.txt', 'w', encoding='UTF-8') as f:
    f.write(str(black_list))

In [3]:
# Фиксируем изменения
connection.commit()
# проверка кол-ва записей в БД
pd.read_sql("SELECT COUNT(*) FROM telecom_companies", connection)

,COUNT(*)
0,20917


In [5]:
# Закрываем соединение
connection.close()

In [4]:
df_full

,ogrn,inn,kpp,name,full_name,data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД
0,1114910000480,4909109443,490901001,"ООО ""ИТЛАЙН""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИТЛАЙН""",61.10
1,1127847488053,7811531478,781101001,"ООО ""СМО СИСТЕМА""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СМО ...",61.30
2,1197536005787,7536178239,753601001,"ООО ""ИНТЕР КОД""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИНТЕ...",61.90
3,1197536005919,7513007283,751301001,"ООО ""ВОНГОЛА""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОНГ...",61.10
4,1197627000691,7604352760,760401001,"ООО ""РЭС""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РИНГ...",61.10
...,...,...,...,...,...,...
20912,1111902001694,1902024322,190201001,"ООО ""САЯНСВЯЗЬ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САЯН...",61.10.1
20913,1112031001422,2016001613,201601001,"ООО ""АВТО-НАВИГАТОР""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АВТО...",61.1
20914,1112031002038,2014005080,201401001,"ООО ""ГАРАНТТОРГ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГАРА...",61.10
20915,1112031003831,2014005877,201401001,"ООО ""РЕАС""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕАС""",61.10


In [6]:
df_full.to_csv('df_full.csv')